## Hypothesis Test Example

In [1]:
//using namespace Roostats;
RooStats::HypoTestResult * result = nullptr;
RooStats::TestStatistic * testStat = nullptr; 
RooStats::ToyMCSampler * toymcs = nullptr; 
RooStats::HypoTestPlot * plot = nullptr; 

First part is just to access the workspace file and retrieve the model and the data 

In [2]:
TString fileName ="HiggsModel_polbkg.root";  
//TString fileName ="CountingModel.root";  
TString workspaceName = "w";
TString modelConfigName = "ModelConfig";
TString dataName = "bindata";
TString integrationType = "";  
ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");

In [3]:
auto file = TFile::Open(fileName);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [4]:
auto w =  (RooWorkspace*) file->Get(workspaceName);
w->Print();
auto sbModel = (RooStats::ModelConfig*) w->obj(modelConfigName);
auto  data = w->data(dataName);
auto poi = (RooRealVar*) sbModel->GetParametersOfInterest()->first();


RooWorkspace(w) w contents

variables
---------
(a1,a2,b0,b1,b2,mass,nbackground,nsignal,width,x)

p.d.f.s
-------
RooExponential::bmodel[ x=z c=1 ] = 0.000616625
RooChebychev::bmodel_pol[ x=x coefList=(b0,b1,b2) ] = 0.802362
RooAddPdf::model[ nbackground * bmodel_pol + nsignal * smodel ] = 0.796066
RooGaussian::smodel[ x=x mean=mass sigma=width ] = 3.47579e-14

functions
--------
RooFormulaVar::z[ actualVars=(a1,a2,x) formula="-(a1*x/100+a2*(x/100)^2)" ] = -7.39125

datasets
--------
RooDataSet::data(x)

named sets
----------
ModelConfig_NuisParams:(a1,a2,nbackground)
ModelConfig_Observables:(x)
ModelConfig_POI:(nsignal)
nuisParams:(a1,a2,nbackground)

generic objects
---------------
RooStats::ModelConfig::ModelConfig



##### Make the b Model by cloning the b model and use a value = 0 for the parameter of interest

In [5]:
auto bModel = (RooStats::ModelConfig*) sbModel->Clone();
poi->setVal(0);
bModel->SetSnapshot( *poi  );
bModel->SetName("B Model");
sbModel->SetName("S+B Model");
sbModel->Print();
bModel->Print();


=== Using the following for S+B Model ===
Observables:             RooArgSet:: = (x)
Parameters of Interest:  RooArgSet:: = (nsignal)
Nuisance Parameters:     RooArgSet:: = (a1,a2,nbackground)
PDF:                     RooAddPdf::model[ nbackground * bmodel_pol + nsignal * smodel ] = 0.802362


=== Using the following for B Model ===
Observables:             RooArgSet:: = (x)
Parameters of Interest:  RooArgSet:: = (nsignal)
Nuisance Parameters:     RooArgSet:: = (a1,a2,nbackground)
PDF:                     RooAddPdf::model[ nbackground * bmodel_pol + nsignal * smodel ] = 0.802362
Snapshot:                
  1) 0x7fb849d06780 RooRealVar:: nsignal = 0 +/- 92.1548  L(0 - 1000)  "nsignal"



In [6]:
// RooStats::AsymptoticCalculator::SetPrintLevel(-1);  // to switch off print level 
RooStats::AsymptoticCalculator  asymCalc(*data, *sbModel, *bModel);

[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] ERROR:InputArguments -- AsymptoticCalculator::Initialize - data set has not been defined


Configure the calculator

In [ ]:
asymCalc.SetOneSidedDiscovery(true);  // for one-side discovery test
//asymCalc.SetPrintLevel(-1);  // to suppress print level 

Run the calculator and get the result

In [ ]:
result = asymCalc.GetHypoTest();
result->Print();

In [ ]:
std::cout << "Significance = " << result->Significance() << " for p-value = " << result->NullPValue() << std::endl; 

### Frequentist Calculator

We run now on the same model the FrequentistCalculator. The Frequentist Calculator uses the test statistic distributions obtained with pseudo-experiments.

In [ ]:
RooStats::FrequentistCalculator   fc(*data, *sbModel, *bModel);

We configure the Frequentist calculator by specifying the number of toys for the two hypothesis 

In [ ]:
fc.SetToys(5000,2000);    // 2000 for null (B) and 500 for alt (S+B) 

We need also to specify the test statistics type. Here are some possible test statistics to use 

In [ ]:
testStat = new RooStats::ProfileLikelihoodTestStat(*sbModel->GetPdf());
// needed for PL test statistics
if (dynamic_cast<RooStats::ProfileLikelihoodTestStat *>(testStat))
   ((RooStats::ProfileLikelihoodTestStat *)testStat)->SetOneSidedDiscovery(true);

In [ ]:
toymcs = (RooStats::ToyMCSampler*)fc.GetTestStatSampler();
toymcs->SetTestStatistic(testStat);

In [ ]:
// for number counting experiments (i.e. when we have only one event per toy)
// in general shape cases are extended model
if (!sbModel->GetPdf()->canBeExtended())
    toymcs->SetNEventsPerToy(1);

Run now the calculator. It can take some time... be patient 

In [ ]:
result = fc.GetHypoTest(); 
result->Print();

Plot now the test statistics distributions

In [ ]:
plot = new RooStats::HypoTestPlot(*result);
plot->SetLogYaxis(true);
plot->Draw();
gPad->Draw();